Connor Krill July 22, 2019
# Visualize the hyades output data as movies!
Plots lagranian distance vs variable and animates it over the entire simulation time.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import animation
from ipywidgets import HTML
import os
import numpy as np
import json

plt.style.use('ggplot')

def animate(i):
    '''Updates the information shown on the axis.
        Relies on externally defined variables.'''
#     txt = f'Iteration: {i}'
#     my_text._text = txt
    txt = f'Optimization of Tin Pressure Drive {str(i).zfill(3)}/{frame_num}'
    ax1.set_title(txt, fontsize=22)
    
    x = json_data['iterations'][str(i).zfill(3)]['pres_time']
    y = json_data['iterations'][str(i).zfill(3)]['pres']
    lines[0].set_data(x, y)
    
    lines[1].set_data(range(i), residuals[:i])
#     ax2.set_xlim(0, i+1)
    return lines

In [2]:
path = './data' #'/Users/krill2/Documents/HyadesOptimizer/data/'
run_name = 'Tin_aug2A'
# label_material = 'Chromium'

save_movie = True
out_fname = 'Tin_aug2 Optimization Restart'

# Setup - Only need to edit the cell below. 
Adjustments for figure lables and movie properities can be made further down in the code.

In [3]:
json_file = os.path.join(path, run_name, f'{run_name}_optimization.json')
with open(json_file) as f:
    json_data = json.load(f)
    
# frame_num = 50
frame_num = len(json_data['iterations']) # the number of time steps is the number of frames  
residuals = [json_data['iterations'][str(i).zfill(3)]['residual'] for i in range(frame_num)]


fig = plt.figure(figsize=(8,8))
ax1 = plt.subplot(3,1,(1,2))
ax2 = plt.subplot(3,1,3)

# add lines to axis
lines = []
x = json_data['iterations']['000']['pres_time']
y = json_data['iterations']['000']['pres']
L, = ax1.plot(x, y, 'bo-', markersize=10, lw=6, alpha=1) 
lines.append(L)
L, = ax2.semilogy([0], json_data['iterations']['000']['residual'], color='orange', lw=6)
lines.append(L)

max_pres = 0
for i in range(len(json_data['iterations'])):
    i_max = max(json_data['iterations'][str(i).zfill(3)]['pres'])
    if i_max > max_pres:
        max_pres = i_max

# format the plot
ax1.set_title(f'Optimization of Tin Pressure Drive 000/{frame_num}', fontsize=22)
ax1.set_xlabel('Nanoseconds', fontsize=22)
ax1.set_ylabel('Pressure Input (GPa)', fontsize=22, ) #color='b')
ax1.set(ylim=(0, max_pres+5))
ax1.set_xlim(0,)

ax1.tick_params(axis='both', labelsize=18)
ax2.tick_params(axis='both', labelsize=18)

length = len(json_data['iterations'])
ax2.set(xlim=(0, length), ylim=(1e-0, max(residuals) + 1e2))
ax2.set_xlabel('Iteration Number',fontsize=22)
ax2.set_ylabel('Residual', fontsize=22)

# call the animator
anim = animation.FuncAnimation(fig, animate,
                               frames=frame_num, interval=200,
                               repeat_delay=2000, blit=True)
plt.close() # close the figure automatically generated by the animiation call
fig.tight_layout()

# optional saving
if save_movie:
    print('Saving...')
    out_path = './'
    Writer = animation.writers['ffmpeg']
    writer = Writer(fps=20, metadata=dict(artist='Me'), bitrate=1800)
    anim.save(os.path.join(out_path, out_fname+'.mp4'), dpi=200, writer=writer)
    print('Saved {}'.format(out_fname))
else:
    print('Displaying Movie...')
HTML(anim.to_html5_video()) # defaults to 20 embed_limit=400.0

Saving...
Saved Tin_aug2 Optimization Restart


HTML(value='<video width="576" height="576" controls autoplay loop>\n  <source type="video/mp4" src="data:vide…